## Experimento 11  — Comparação de Modelos + GridSearchCV

**Objetivo didático:** mostrar duas etapas complementares de seleção de modelos:
1) **Parte 1 (Baseline, sem GridSearch):** comparar modelos com *defaults* bem escolhidos, usando `Pipeline` e validação cruzada estratificada.
2) **Parte 2 (Com GridSearchCV):** sintonizar hiperparâmetros dos **mesmos classificadores**, comparar por CV e avaliar no teste, salvando o melhor.

> No *Iris*, modelos lineares costumam ir muito bem. É esperado que o baseline seja forte e, às vezes, até supere o resultado do GridSearch quando o *grid* está conservador.


### 🎯 Objetivos de aprendizagem
- Evitar *data leakage* com `Pipeline` + `StandardScaler` quando apropriado.
- Comparar classificadores sob o mesmo esquema de validação cruzada.
- Entender quando *defaults* já são muito bons.
- Usar `GridSearchCV` de forma consistente com os mesmos modelos do baseline.
- Registrar resultados em tabelas para facilitar a análise.


In [2]:
# ============== Configuração Inicial ==============
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Dados
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Esquema de CV consistente nas duas partes
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)


## Parte 1 — Baseline consistente (sem GridSearch)
- **Mesmos modelos** usados depois no GridSearch: `LogReg`, `SVC (linear)`, `RandomForest`.
- `Pipeline` + `StandardScaler` em LogReg e SVC (evita *data leakage*).
- Comparação por **CV estratificada** (média ± desvio) e avaliação no **teste**.


In [3]:
# Candidatos (sem GridSearch)
candidates = {
    "LogReg": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=2000, random_state=RANDOM_STATE))
    ]),
    "SVC_linear": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="linear", random_state=RANDOM_STATE))
    ]),
    "RF": RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
}

rows = []
for name, model in candidates.items():
    cv_res = cross_validate(
        model, X_train, y_train,
        cv=cv, scoring="accuracy",
        return_train_score=False, n_jobs=-1
    )
    cv_mean = cv_res["test_score"].mean()
    cv_std  = cv_res["test_score"].std()

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)

    rows.append({"model": name, "cv_mean_acc": cv_mean, "cv_std_acc": cv_std, "test_acc": test_acc})

df_base = pd.DataFrame(rows).sort_values(by="cv_mean_acc", ascending=False).reset_index(drop=True)
print(df_base)

best_name = df_base.iloc[0]["model"]
best_model_baseline = candidates[best_name]
print(f"\n== Melhor por CV (sem GridSearch): {best_name} ==")

# Relatório do melhor baseline
best_model_baseline.fit(X_train, y_train)
y_pred_best = best_model_baseline.predict(X_test)
print(f"Acurácia no teste: {accuracy_score(y_test, y_pred_best):.3f}\n")
print("Relatório de classificação:\n", classification_report(y_test, y_pred_best))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred_best))


        model  cv_mean_acc  cv_std_acc  test_acc
0  SVC_linear     0.975000    0.033333  1.000000
1      LogReg     0.958333    0.026352  0.933333
2          RF     0.950000    0.031180  0.900000

== Melhor por CV (sem GridSearch): SVC_linear ==
Acurácia no teste: 1.000

Relatório de classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Matriz de confusão:
 [[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]


## Parte 2 — Sintonização com GridSearchCV (mesmos modelos)
- Mantemos **os mesmos três classificadores** do baseline.
- Usamos **grids enxutos e razoáveis** para não superexplorar o Iris, mas suficientes para demonstrar tuning.
- Critério de seleção: **maior `cv_mean_acc`**.
- Avaliamos no teste e salvamos o **melhor modelo**.
